## load prerequisites

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import csv
import random
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf

In [3]:
print(tf.__version__)

1.15.0


## Preprocess data : names

In [4]:
data_name = set()
#data_name = list()
max_len = 0
with open('./babyK.csv',encoding='CP949') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            tmp_name = row[0]
            data_name.add(row[0])
            if len(tmp_name) > max_len:
                max_len = len(tmp_name)
data_name = list(data_name)
print('name total : {}'.format(len(data_name)))
print('maximum name length : {}'.format(max_len))
print(data_name)

name total : 489
maximum name length : 2
['수민', '도영', '다영', '은재', '채민', '태린', '은별', '예랑', '하늬', '채영', '세영', '새봄', '윤서', '혜리', '가온', '도윤', '채아', '하린', '채윤', '수아', '윤지', '현아', '채령', '소율', '아민', '지나', '가율', '서연', '승연', '민선', '태경', '다현', '은빈', '도은', '아현', '다솔', '한비', '예영', '나윤', '지안', '다경', '연재', '유주', '윤채', '유라', '미나', '서정', '수정', '송현', '혜윤', '시하', '해린', '혜린', '태림', '효림', '재이', '선율', '아율', '인영', '하은', '소정', '은정', '정빈', '리아', '시영', '서진', '유미', '정현', '서현', '서인', '규원', '단비', '서유', '태은', '초아', '정인', '태희', '한나', '예빈', '효주', '다민', '예령', '이현', '예슬', '수린', '세희', '예나', '다연', '희원', '채원', '하나', '도이', '서희', '시유', '소예', '하윤', '준희', '가윤', '소미', '로아', '효진', '승현', '채연', '세은', '유희', '이수', '나은', '라은', '아영', '아린', '태인', '하진', '루리', '유민', '서영', '서율', '혜민', '정민', '하온', '여울', '세연', '희수', '예리', '슬아', '서안', '지선', '혜정', '아림', '소연', '보람', '서빈', '지후', '서이', '주원', '다솜', '누리', '혜진', '고은', '진아', '채은', '하원', '재연', '예은', '인서', '예주', '나율', '담희', '민아', '미소', '세아', '은성', '윤영', '은진', '효빈', '영은', '민서', '리하', '진주', '은비', '다미'

## Preprocess data : Characters

In [6]:
chars = set()
for name in data_name:
    for char in name:
        chars.add(char)
chars = list(np.sort(list(chars)))
print('{} korean : '.format(len(chars)), chars)

90 korean :  ['가', '겸', '경', '고', '규', '나', '누', '늘', '늬', '다', '단', '담', '도', '라', '란', '람', '랑', '래', '레', '려', '령', '로', '루', '름', '리', '린', '림', '미', '민', '별', '보', '봄', '비', '빈', '빛', '사', '새', '서', '선', '설', '성', '세', '소', '솔', '솜', '송', '수', '슬', '승', '시', '아', '안', '엘', '여', '연', '영', '예', '오', '온', '우', '울', '원', '유', '윤', '율', '은', '을', '음', '이', '인', '임', '재', '정', '제', '주', '준', '지', '진', '채', '초', '태', '하', '한', '해', '현', '혜', '호', '효', '후', '희']


## Define function to convert name to onehot

In [7]:
def name_to_onehot(names, chars, max_len):
    onehot = np.zeros((len(names), max_len, len(chars)+1))
    for idx_1, name in enumerate(names):
        for idx_2 in range(max_len):
            if idx_2 < len(name):
                idx_3 = chars.index(name[idx_2])
                onehot[idx_1, idx_2, idx_3] = 1
            else:
                onehot[idx_1, idx_2, -1] = 1
    return onehot

onehot_ex = name_to_onehot(['가'], chars, max_len)
onehot_ex.shape #1은 배치, 다음은 rnn 차원 수

(1, 2, 91)

## Define dimension and Placeholders

In [0]:
num_data = len(data_name)
seq_len = max_len - 1
dim_data = len(chars) + 1

ph_input_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])
ph_output_name = tf.placeholder(dtype=tf.float32, shape=[None, seq_len, dim_data])

In [9]:
print(dim_data)

91


## Define weight variables

In [0]:
#입출력을 위한 fully-connected weighted vector 정의
dim_rnn_cell = 128
stddev = 0.02

with tf.variable_scope('weights'):
    W_i = tf.get_variable('W_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data, dim_rnn_cell],
                         stddev = stddev))
    b_i = tf.get_variable('b_i', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell],
                         stddev = stddev))
    W_o = tf.get_variable('W_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_rnn_cell, dim_data],
                         stddev = stddev))
    b_o = tf.get_variable('b_o', dtype=tf.float32,
                         initializer=tf.random_normal([dim_data],
                         stddev = stddev))

## Define RNN for training

In [0]:
#배열로 입력하되 길이가 1되도록 해서 각각의 값을 넣어줌:개별 값을 넣어주기 위함(_split)
def name_rnn_train(_x, _seq_len, _dim_data, _dim_rnn_cell):
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
        
    # Linear Operation for Input
    _h_split = tf.matmul(_x_split, W_i) + b_i
    _h_split = tf.split(_h_split,_seq_len, axis=0)    
       
    # Define LSTM Cell && RNN 
    with tf.variable_scope('rnn',reuse=tf.AUTO_REUSE):
        _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(_dim_rnn_cell)
        _output, _state = tf.nn.static_rnn(_rnn_cell, _h_split, dtype=tf.float32)
    
    # Linear Operation for Output
    _total_out = []
    for _tmp_out in _output:
      _tmp_out = tf.matmul(_tmp_out, W_o) + b_o
      _total_out.append(_tmp_out)
        
    return tf.transpose(tf.stack(_total_out), [1, 0, 2])

## Define result graph

In [12]:
result_name = name_rnn_train(ph_input_name, seq_len, dim_data, dim_rnn_cell)
print('result_shape :', result_name.shape)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
result_shape : (?, 1, 91)


## Define Loss function

In [0]:
def name_loss(_gt_name, _result_name, _seq_len):
    total_loss = 0
    for i in range(_seq_len):
        gt_char = _gt_name[:,i,:]
        result_char = _result_name[:,i,:]
        tmp_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=gt_char,logits=result_char)
        tmp_loss = tf.reduce_mean(tmp_loss)
        total_loss += tmp_loss
    return total_loss
rnn_loss = name_loss(ph_output_name, result_name, seq_len)

## Define Optimizer and Get Ready

In [14]:
learning_rate = 1e-3
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(rnn_loss)

init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=tf.trainable_variables())

print('Now ready to start the session')

Now ready to start the session


## Session Run

In [15]:
max_epoch = 20
batch_size = 32
num_batch = int(num_data/batch_size)
with tf.Session() as sess:
    sess.run(init)
    
    for _epoch in range(max_epoch):
        random.seed(_epoch)
        batch_shuffle = list(range(num_data))
        random.shuffle(batch_shuffle)
        
        total_train_loss = 0
        
        for i in range(num_batch):
            batch_idx = [batch_shuffle[idx] for idx in range(i*batch_size,
                                                            (i+1)*batch_size)]
            batch_names = [name for name in data_name if data_name.index(name) in batch_idx]
            batch_onehots = name_to_onehot(batch_names, chars, max_len)
            
            input_onehot = batch_onehots[:, 0:(max_len-1), :] #input 들어가는 부분(0번째)
            output_onehot = batch_onehots[:, 1:max_len, :] #output 다시 들어가는 부분(1번째)

            train_feed_dict = {ph_input_name: input_onehot, 
                               ph_output_name: output_onehot}
            
            sess.run(optimizer, feed_dict = train_feed_dict)
            curr_loss = sess.run(rnn_loss, feed_dict=train_feed_dict)
            
            total_train_loss += curr_loss/num_batch

        print('epoch : {}, train_loss : {}'.format(_epoch+1, total_train_loss))
        
        model_save_path = saver.save(sess, './RNN_model/model.ckpt', global_step=_epoch+1)
        print('Model saved in file: {}'.format(model_save_path))

epoch : 1, train_loss : 4.498014354705811
Model saved in file: ./RNN_model/model.ckpt-1
epoch : 2, train_loss : 4.4341453870137535
Model saved in file: ./RNN_model/model.ckpt-2
epoch : 3, train_loss : 4.163856013615927
Model saved in file: ./RNN_model/model.ckpt-3
epoch : 4, train_loss : 3.8410646756490068
Model saved in file: ./RNN_model/model.ckpt-4
epoch : 5, train_loss : 3.782847722371419
Model saved in file: ./RNN_model/model.ckpt-5
epoch : 6, train_loss : 3.745732100804647
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Model saved in file: ./RNN_model/model.ckpt-6
epoch : 7, train_loss : 3.731955337524414
Model saved in file: ./RNN_model/model.ckpt-7
epoch : 8, train_loss : 3.7238747755686443
Model saved in file: ./RNN_model/model.ckpt-8
epoch : 9, train_loss : 3.7095814387003587
Model saved in file: ./RNN_model/model.ckpt-9
epoch : 10, train_loss : 3.701665306091309
Model saved in file: ./RNN_model/model.ckpt-10
epoch : 11, train_loss : 3.693

## Define RNN to test

In [0]:
ph_test_input_name = tf.placeholder(dtype=tf.float32, shape=[1, 1, dim_data])
ph_h = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])
ph_c = tf.placeholder(dtype=tf.float32, shape=[1, dim_rnn_cell])

def name_rnn_test(_x, _dim_data, _dim_rnn_cell, _prev_h, _prev_c):
    _x_split = tf.transpose(_x, [1, 0, 2]) # seq_len, batch, dim_data
    _x_split = tf.reshape(_x_split, [-1, _dim_data])
    
    # Linear Operation for Input
    _h_split = tf.matmul(_x_split, W_i) + b_i
    _h_split = tf.split(_h_split,1,axis=0) #1 is the seq_len

    # Define LSTM Cell && RNN including (h, c)
    with tf.variable_scope('rnn',reuse=tf.AUTO_REUSE):
      _rnn_cell = tf.nn.rnn_cell.BasicLSTMCell(_dim_rnn_cell)
      _output, _state = tf.nn.static_rnn(_rnn_cell,_h_split,dtype=tf.float32,initial_state=(_prev_h,_prev_c))
    
    # Linear Operation for Ouput
    _total_out = []    
    for _tmp_out in _output:
      _tmp_out = tf.matmul(_tmp_out,W_o) + b_o
      _total_out.append(_tmp_out)
    return tf.transpose(tf.stack(_total_out),[1,0,2]),_state

## Run Test

In [18]:
test_result_name, test_state = name_rnn_test(ph_test_input_name, dim_data, dim_rnn_cell,ph_h, ph_c)

with tf.Session() as sess:
  sess.run(init)
  for i in range(100):
    saver.restore(sess, './RNN_model/model.ckpt-20')
    
    total_name = ''
    prev_char = str(np.random.choice(chars))
    total_name += prev_char
    prev_state = (np.zeros((1, dim_rnn_cell)), np.zeros((1, dim_rnn_cell)))
    for i in range(seq_len):
        input_onehot = np.zeros((1, 1, dim_data))
        prev_char_idx = chars.index(prev_char)
        input_onehot[:, :, prev_char_idx] = 1
            
        test_feed_dict = {ph_test_input_name: input_onehot,
                          ph_h: prev_state[0], ph_c: prev_state[1]}
        curr_result, curr_state = sess.run([test_result_name, test_state], test_feed_dict)
        if np.argmax(curr_result) == dim_data-1: #확률 가장 높은 것 있으면 중지
            break
        else: #아닌경우 hidden state, cell state 바꿔가며 계산
            softmax_result = sess.run(tf.nn.softmax(test_result_name), test_feed_dict)
            softmax_result = np.squeeze(softmax_result)
            softmax_result = softmax_result[:dim_data-1]/sum(softmax_result[:dim_data-1])
            prev_char = np.random.choice(chars,1,p=softmax_result)
            total_name += prev_char[0]
            prev_state = curr_state    
    print('새로운 아기이름 :', total_name)

INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 세솔
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 누아
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 나이
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 제람
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 서은
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 보은
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 윤린
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 비예
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 재연
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 한설
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 누하
INFO:tensorflow:Restoring parameters from ./RNN_model/model.ckpt-20
새로운 아기이름 : 랑아
INFO:tensorflow:

In [0]:
prev_char=chars.pop()

In [0]:
 prev_char = np.random.choice(chars)

In [0]:
type(str(prev_char))

str

In [0]:
print(prev_char)

욱
